In [27]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from haversine import haversine
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [28]:
injuries = pd.read_csv("NBA_Injury_Data.csv")
df_cities = pd.read_csv("stadiums.csv") 
df_cities = df_cities[df_cities['League'] == 'NBA']
player = pd.read_csv("joel_embiid_2020_2021.csv")
name = 'Joel Embiid'

In [29]:
daysBetweenRegressor = 0 #calculate game and day played frequency 
minsPlayedRegressor = 0 #take into account mins played yesterday
distTraveledRegressor = 0 #distance traveled to play
injuryRegressor = 0 #effect and severity of injury 
currentInjury = None #name of injury

In [30]:
teams_expanded = {'DAL': 'Dallas Mavericks', 
                'ORL': 'Orlando Magic', 
                'SAS': 'San Antonio Spurs',
                'DEN': 'Denver Nuggets',
                'BKN': 'Brooklyn Nets',
                'BRK': 'Brooklyn Nets',
                'UTA': 'Utah Jazz',
                'WAS': 'Washington Wizards',
                'GSW': 'Golden State Warriors',
                'LAC': 'Los Angeles Clippers',
                'LAL': 'Los Angeles Lakers',
                'MEM': 'Memphis Grizzlies', 
                'MIL': 'Milwaukee Bucks',
                'PHX': 'Phoenix Suns',
                'PHO': 'Phoenix Suns',
                'MIA': 'Miami Heat',
                'IND': 'Indiana Pacers',
                'SAC': 'Sacremento Kings',
                'DET': 'Detroit Pistons',
                'PHI': 'Philadelphia 76ers', 
                'NYK': 'New York Knicks',
                'POR': 'Portland Trail Blazers',
                'OKC': 'Oklahoma City Thunder', 
                'CLE': 'Cleveland Cavaliers',
                'TOR': 'Toronto Raptors',
                'NOP': 'New Orleans Pelicans',
                'CHO': 'Charlotte Hornets',
                'ATL': 'Atlanta Hawks',
                'MIN': 'Minnesota Timberwolves',
                'BOS': 'Boston Celtics',
                'HOU': 'Houston Rockets',
                'CHI': 'Chicago Bulls',
                }

In [31]:
'''
https://towardsdatascience.com/realigning-sports-leagues-with-a-clustering-algorithm-d6e9de9294d0
'''
def dist(lat1, long1, lat2, long2):
    
    lat1, lat2 = np.deg2rad(lat1), np.deg2rad(lat2)
    long1, long2 = np.deg2rad(long1), np.deg2rad(long2)
    EARTH_RADIUS = 3958.8
    return EARTH_RADIUS * np.arccos((np.sin(lat1) * np.sin(lat2)) + 
                          np.cos(lat1) * np.cos(lat2) * np.cos(long2 - long1))

In [32]:
def calcDist(start, end):
    print(start, end)
    start_long = df_cities.loc[df_cities.Team == teams_expanded[start], 'Long'].tolist()[0]
    start_lat = df_cities.loc[df_cities.Team == teams_expanded[start], 'Lat'].tolist()[0]
    end_long = df_cities.loc[df_cities.Team == teams_expanded[end], 'Long'].tolist()[0]
    end_lat = df_cities.loc[df_cities.Team == teams_expanded[end], 'Lat'].tolist()[0]
    print(haversine((start_lat, start_long),(end_lat, end_long), unit='mi'))
    return haversine((start_lat, start_long),(end_lat, end_long), unit='mi')

    

In [33]:
def regressionCalc(injury):
    print("injury detected")
    print(injury)
    currInj = 'standard'
    injRegress = -0.10
    if '(out for season)' in injury:
        currInj = 'EOS'
        injRegress = -1
    elif 'sore' in injury:
        currInj = 'soreness' 
        injRegress = -0.10
    elif 'surgery' in injury:
        currInj = 'surgery recovery' 
        injRegress = -0.10
    elif 'injury' in injury:
        currInj = 'standard injury'
        injRegress = -0.10

    return injRegress

    


In [34]:
def getDayDiff(day1, day2): 
    x = datetime.strptime(day1, "%Y-%m-%d")
    y = datetime.strptime(day2, "%Y-%m-%d")
    return (x-y).days

In [35]:
prevLoc = 'NOP'
prevMins = 0
prevDate = player['Date'].iloc[0]
print(prevDate)
if player['Date'].iloc[0] == None:
    print(player['Home'].iloc[0])
else: 
    print(player['Opp'].iloc[0])



2020-12-23
WAS


In [36]:
cnt = 0
values = []
values.append(1)
for row in player.itertuples():
    print(row.MP[0:2])
    if row.MP[0:2] != 'In' and row.MP[
    0:2] != 'Di':
        if values[-1] == 0.6: 
            values[-1] = 0.8
        if row.Home == '@':
            dist = calcDist(prevLoc, row.Opp)
            prevLoc = row.Opp
        else:
            dist = calcDist(prevLoc, row.Tm)
            prevLoc = row.Tm


        days = getDayDiff(row.Date, prevDate)
        prevDate = row.Date
        minutes = row.MP[0:2]

        result = injuries[(injuries['Relinquished'] == name) & (injuries['Date'] == row.Date)]
        if result.empty == False:
            injury = regressionCalc(result.Notes)
        else: 
            injury = 0
        print("Distance Traveled: " , dist)
        print("Injury Value: " , injury)
        print("Days Rest: " , days)

        if dist == 0:
            distRegress = 0.10
        elif dist < 1000:
            distRegress = -0.10
        elif dist < 2000:
            distRegress = -0.15
        else: 
            distRegress = -0.20
        if days == 1:
            daysBonus = 0
        elif days == 2:
            daysBonus = 0.05
        elif days == 3: 
            daysBonus = 0.10
        else:
            daysBonus = 0.15
        if re.search('[a-zA-Z]', minutes) == False:
            if int(minutes) < 20:
                minsRegressor = 0
            elif int(minutes) < 30:
                minsRegressor = -0.10
            elif int(minutes) < 40:
                minsRegressor = -0.15
            else: 
                minsRegressor = -0.20
        else:
            minsRegressor = 0
        
        healthValue = values[-1] + (0.33 *  distRegress) + (0.33 *  minsRegressor) + (0.33 * daysBonus) + injury
        if healthValue >= float(1):
            values.append(1.0)
        else:
            values.append(healthValue)
        print(values[-1])
        cnt += 1
    else:
        values.append(0.6)
        print(0.6)

34
NOP PHI
1086.5167688743338
Distance Traveled:  1086.5167688743338
Injury Value:  0
Days Rest:  0
1.0
30
PHI NYK
85.4219723416408
Distance Traveled:  85.4219723416408
Injury Value:  0
Days Rest:  3
1.0
Di
0.6
37
NYK PHI
85.4219723416408
Distance Traveled:  85.4219723416408
Injury Value:  0
Days Rest:  3
0.8
26
PHI ORL
860.9286607753394
Distance Traveled:  860.9286607753394
Injury Value:  0
Days Rest:  2
0.7835
36
ORL PHI
860.9286607753394
Distance Traveled:  860.9286607753394
Injury Value:  0
Days Rest:  2
0.7669999999999999
24
PHI PHI
0.0
Distance Traveled:  0.0
Injury Value:  0
Days Rest:  2
0.8164999999999999
36
PHI PHI
0.0
Distance Traveled:  0.0
Injury Value:  0
Days Rest:  2
0.8659999999999999
29
PHI BRK
83.02682705930822
Distance Traveled:  83.02682705930822
Injury Value:  0
Days Rest:  1
0.8329999999999999
In
0.6
22
BRK ATL
745.854611292941
Distance Traveled:  745.854611292941
Injury Value:  0
Days Rest:  4
0.8165
39
ATL PHI
663.0112509311847
Distance Traveled:  663.011250931

In [37]:
print(values)
values = values[1:]
print(values)
player['Injury and Fatigue Score'] = values

[1, 1.0, 1.0, 0.8, 0.8, 0.7835, 0.7669999999999999, 0.8164999999999999, 0.8659999999999999, 0.8329999999999999, 0.8, 0.8165, 0.7835, 0.833, 0.8, 0.8825000000000001, 0.932, 0.899, 0.8, 0.8165, 0.7999999999999999, 0.8, 0.8165, 0.7835, 0.833, 0.7999999999999999, 0.7834999999999999, 0.7504999999999998, 0.8, 0.7835, 0.833, 0.8164999999999999, 0.8659999999999999, 0.8494999999999998, 0.8989999999999998, 0.9484999999999998, 0.9979999999999998, 0.8, 0.8165, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.8165, 0.8, 0.8, 0.7835000000000001, 0.7505000000000001, 0.734, 0.701, 0.7505, 0.8165, 0.866, 0.833, 0.8, 0.8165, 0.866, 0.9155, 0.8825, 0.833, 0.8164999999999999, 0.7834999999999999, 0.8164999999999999, 0.8, 0.8, 0.7505000000000001, 0.6]
[1.0, 1.0, 0.8, 0.8, 0.7835, 0.7669999999999999, 0.8164999999999999, 0.8659999999999999, 0.8329999999999999, 0.8, 0.8165, 0.7835, 0.833, 0.8, 0.8825000000000001, 0.932, 0.899, 0.8, 0.8165, 0.7999999999999999, 0.8, 0.8165, 0.7835, 0.833, 0.7999999999999999, 

In [38]:
print(player['Injury and Fatigue Score'])

0     1.0000
1     1.0000
2     0.8000
3     0.8000
4     0.7835
       ...  
67    0.8165
68    0.8000
69    0.8000
70    0.7505
71    0.6000
Name: Injury and Fatigue Score, Length: 72, dtype: float64
